<a href="https://colab.research.google.com/github/dgmyaing/labs/blob/main/HedonicPricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/DS3001/labs/refs/heads/main/04_hedonic_pricing/airbnb_hw.csv'
data = pd.read_csv(url)

print("General Info:")
data.info()

print(data.head())

#below is data cleaning
print(data.isnull().sum())
#Seeing if there are any values missing and cleaning from there
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
#Getting a little summary set below
print("\nSummary Stats:")
print(data.describe())


General Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30478 entries, 0 to 30477
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Host Id                     30478 non-null  int64  
 1   Host Since                  30475 non-null  object 
 2   Name                        30478 non-null  object 
 3   Neighbourhood               30478 non-null  object 
 4   Property Type               30475 non-null  object 
 5   Review Scores Rating (bin)  22155 non-null  float64
 6   Room Type                   30478 non-null  object 
 7   Zipcode                     30344 non-null  float64
 8   Beds                        30393 non-null  float64
 9   Number of Records           30478 non-null  int64  
 10  Number Of Reviews           30478 non-null  int64  
 11  Price                       30478 non-null  object 
 12  Review Scores Rating        22155 non-null  float64
dtypes: float64(4), in

In [12]:
df.head()
#Felt like this may give a cleaner view of what I'm looking at

,Host Id,Host Since,Name,Neighbourhood,Property Type,Review Scores Rating (bin),Room Type,Zipcode,Beds,Number of Records,Number Of Reviews,Price,Review Scores Rating
0,5162530,NaN,1 Bedroom in Prime Williamsburg,Brooklyn,Apartment,NaN,Entire home/apt,11249.0,1.0,1,0,145,NaN
1,33134899,NaN,"Sunny, Private room in Bushwick",Brooklyn,Apartment,NaN,Private room,11206.0,1.0,1,1,37,NaN
2,39608626,NaN,Sunny Room in Harlem,Manhattan,Apartment,NaN,Private room,10032.0,1.0,1,1,28,NaN
3,500,6/26/2008,Gorgeous 1 BR with Private Balcony,Manhattan,Apartment,NaN,Entire home/apt,10024.0,3.0,1,0,199,NaN
4,500,6/26/2008,Trendy Times Square Loft,Manhattan,Apartment,95.0,Private room,10036.0,3.0,1,39,549,96.0


In [17]:
#Struggling with the regression. Orginal input was having trouble with dummy variables and now it's giving me this error
from sklearn import linear_model
from sklearn.model_selection import train_test_split

y = df['Price']
X = df.loc[:,['Review Scores Rating', 'Neighbourhood '] ]
X_train, X_test, y_train, y_test = train_test_split(X,y, # Feature and target variables
                                                    test_size=.2, # Split the sample 80 train/ 20 test
                                                    random_state=100) # For replication purposes

# Impute missing values in 'Review Scores Rating' using the mean
X_train['Review Scores Rating'].fillna(X_train['Review Scores Rating'].mean(), inplace=True)
X_test['Review Scores Rating'].fillna(X_train['Review Scores Rating'].mean(), inplace=True) # Use train mean for test

# Create dummy variables, handling unseen categories
Z_train = pd.concat([X_train['Review Scores Rating'],
                     pd.get_dummies(X_train['Neighbourhood '], dtype='int', handle_unknown='ignore')], axis = 1)
Z_test = pd.concat([X_test['Review Scores Rating'],
                    pd.get_dummies(X_test['Neighbourhood '], dtype='int', handle_unknown='ignore')], axis = 1)

# Align columns between train and test sets to ensure they have the same features
Z_train, Z_test = Z_train.align(Z_test, join='inner', axis=1, fill_value=0)

reg = linear_model.LinearRegression(fit_intercept=False).fit(Z_train,y_train) # Run regression

y_hat = reg.predict(Z_test)
print('Rsq: ', reg.score(Z_test,y_test)) # R2
rmse = np.sqrt( np.mean( (y_test - y_hat)**2 ))
print('RMSE: ', rmse) # R2

results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_}) # Regression coefficients
results

<ipython-input-17-1fe70a06ae39>:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train['Review Scores Rating'].fillna(X_train['Review Scores Rating'].mean(), inplace=True)
<ipython-input-17-1fe70a06ae39>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

TypeError: get_dummies() got an unexpected keyword argument 'handle_unknown'

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import numpy as np


features = ['Property Type', 'Room Type', 'Beds', 'Number of Records', 'Number Of Reviews', 'Review Scores Rating']
X = data[features]
y = data['Price']




In [10]:
#Used Chat to try and help but resulted in this error, unsure of how to fix because Colab isn't accepting my adjusted code
data['Price'] = data['Price'].replace(',', '', regex=True).astype(float)

X = data[features]
y = data['Price']

X = pd.get_dummies(X, columns=['Property Type', 'Room Type'], drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

simple_model = LinearRegression()
simple_model.fit(X_train, y_train)

y_train_pred_simple = simple_model.predict(X_train)
y_test_pred_simple = simple_model.predict(X_test)

rmse_train_simple = mean_squared_error(y_train, y_train_pred_simple, squared=False)
rmse_test_simple = mean_squared_error(y_test, y_test_pred_simple, squared=False)
r2_train_simple = r2_score(y_train, y_train_pred_simple)
r2_test_simple = r2_score(y_test, y_test_pred_simple)

print("Simple Model Results:")
print(f"Training RMSE: {rmse_train_simple:.2f}, Training R^2: {r2_train_simple:.2f}")
print(f"Test RMSE: {rmse_test_simple:.2f}, Test R^2: {r2_test_simple:.2f}")


Simple Model Results:
Training RMSE: 130.69, Training R^2: 0.22
Test RMSE: 133.11, Test R^2: 0.25


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [19]:

X_train['Beds_Squared'] = X_train['Beds'] ** 2
X_test['Beds_Squared'] = X_test['Beds'] ** 2

complex_model = LinearRegression()
complex_model.fit(X_train, y_train)

y_train_pred_complex = complex_model.predict(X_train)
y_test_pred_complex = complex_model.predict(X_test)

rmse_train_complex = mean_squared_error(y_train, y_train_pred_complex, squared=False)
rmse_test_complex = mean_squared_error(y_test, y_test_pred_complex, squared=False)
r2_train_complex = r2_score(y_train, y_train_pred_complex)
r2_test_complex = r2_score(y_test, y_test_pred_complex)

print("\nComplex Model Results:")
print(f"Training RMSE: {rmse_train_complex:.2f}, Training R^2: {r2_train_complex:.2f}")
print(f"Test RMSE: {rmse_test_complex:.2f}, Test R^2: {r2_test_complex:.2f}")


KeyError: 'Beds'

Had trouble with this lab. In all honesty, I forgot it was due tonight and am unable to complete it to the best of my ability. I'll continue work on it tomorrow for late credit if allowed.